In [38]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from numpy.fft import fft, ifft
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
#Components used for signal - ACF,PSD
import scipy.signal

In [75]:
# Getting current working directory and storing it as a string variable and reading all empty seat readings into one dataframe

directory_in_str = "E:\\Haritha\\MasterThesis\\ModelTraining\\aa_measurements\\FFTfilegenerationpurpose\\Part7"
Emptyfilelist = []
empty = pd.DataFrame()
print("Adding the following files : ")
for file in os.listdir(directory_in_str) :
    if file.startswith("adc_empty") :
        print(file)
        filepathempty = directory_in_str +"\\" + file
        df1 = pd.read_csv(filepathempty, sep = '\t', engine = 'python', header = None)
        empty  = empty.append(df1)
    else :
        continue  
 
empty["Label"] = 0
#empty.head()

Adding the following files : 
adc_empty112_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\1321238430.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty  = empty.append(df1)


adc_empty113_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\1321238430.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty  = empty.append(df1)


adc_empty114_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\1321238430.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty  = empty.append(df1)


adc_empty115_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\1321238430.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empty  = empty.append(df1)


In [76]:
# Empty seat Data ingest
empty.reset_index(inplace = True)

#Dropping anomalies
empty.drop(empty[empty.iloc[:,10]>=0.41].index,inplace = True)

#Index given is 16, as iloc uses n-1 as the end limit for 0:16 
empty.drop(empty.iloc[:,0:17] ,axis=1,inplace=True)
#empty.head()


In [77]:
# Reading all baby seat readings into one dataframe

baby = pd.DataFrame()
print("Adding the following files : ")
for file in os.listdir(directory_in_str) :
    if file.startswith("adc_baby") :
        print(file)
        filepathbaby = directory_in_str +"\\" +file
        df2 = pd.read_csv(filepathbaby, sep = '\t', engine = 'python',header = None)
        baby  = baby.append(df2)
    else :
        continue 

baby["Label"] = 1
#baby.head()

Adding the following files : 
adc_baby112_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\3438902681.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baby  = baby.append(df2)


adc_baby113_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\3438902681.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baby  = baby.append(df2)


adc_baby114_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\3438902681.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baby  = baby.append(df2)


adc_baby115_.txt


C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\3438902681.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baby  = baby.append(df2)


In [78]:
#Dropping anomalies
baby.drop(baby[baby.iloc[:,10]>=0.41].index,inplace = True)
baby.reset_index(inplace = True)

#Index given is 16, as iloc uses n-1 as the end limit for 0:16 
# Baby seat Data ingest and attach Label = 1
baby.drop(baby.iloc[:,0:17] ,axis=1,inplace=True)


#baby.head()

In [79]:
total = pd.concat([empty,baby],ignore_index=True)
total.reset_index(drop=True,inplace=True)
#total

In [80]:
total["Mean"]=total.iloc[:,0:16384].mean(axis=1)
temp = total["Mean"]
#total.head()

In [81]:
totalidxnew = total.iloc[:,0:16384]
#totalidxnew

In [82]:
totalfresh = totalidxnew.sub(temp, axis=0)
#totalfresh.head()

In [83]:
total2 = pd.DataFrame()
total2 = totalfresh.iloc[:,0:16384]
total2.reset_index(drop=True,inplace=True)
#total2.head()

In [84]:
temp = total2.iloc[:,0:16384].max(axis=1)
temp.max()

2028.9697875976562

In [85]:
total2["Label"] =  total["Label"]
#total2

C:\Users\Vinoth\AppData\Local\Temp\ipykernel_10308\3584131883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total2["Label"] =  total["Label"]


In [86]:
windowtorawadcdatalength = 16384

In [87]:
adcwindow = np.hamming(windowtorawadcdatalength)

In [88]:
windowedadcsignal = total2.iloc[:,0:16384] * adcwindow
#plt.plot(windowedadcsignal.iloc[397,:])

In [89]:
#plt.plot(total2.iloc[397,0:16384])

In [90]:
lengthvar = len(total2)
#lengthvar

In [91]:
positiveacf = pd.DataFrame(acf(windowedadcsignal.iloc[i,0:16384], nlags = 16383) 
                   for i in range(lengthvar)
                  )

#Note down the time of execution for this step

In [92]:
positiveacf["Label"] = total2["Label"]

In [93]:
#positiveacf

In [94]:
#Reversing the dataframe to get the negative ACF portion
negativeacf = positiveacf[positiveacf.columns[::-1]]

#Deleting the last column (corresponding to lag =0) as it is already present in the positive portion of the ACF
negativeacf = negativeacf.iloc[: , :-1]

In [95]:
#negativeacf

In [96]:
wholeacfsignal = pd.concat([negativeacf, positiveacf], axis=1)
wholeacfsignal.columns = range(wholeacfsignal.columns.size)

In [97]:
wholeacfsignal.iloc[:,1:32768].shape

(1600, 32767)

In [98]:
#plt.plot(wholeacfsignal.iloc[0,1:32768])

In [99]:
X_fft = np.fft.fft(wholeacfsignal.iloc[:,1:32768])
N = 32768
X_fftmagpsd = np.abs(X_fft)/N

#Storing the above numpy array as a dataframe
X_fftpsd = pd.DataFrame(X_fftmagpsd)

In [100]:
X_fftpsd = X_fftpsd.iloc[:,600:1000]
#X_fftpsd

In [101]:
X_fftpsd["Label"] = total2["Label"]
#X_fftpsd

In [102]:
X_fftpsd.to_csv('PSDfromWindowedADC-Part7.csv', encoding='utf-8',index=False)

In [124]:
#Sampling frequency
Fs = 1953125
#Number of samples
N = 32767

#Spectrum only plotted for half of spectrum, thus N/2
f1 = np.linspace(0, 16383*(Fs/N), int(N/2))
f2 = np.linspace(0,8192*(Fs/(N/2)), int(N/4))

#f = f.reshape((1, -1)) 
f.shape

(16383,)